# Configuration

In [1]:
!/emr/notebook-env/bin/pip3 install --upgrade -i https://mirrors.ustc.edu.cn/pypi/web/simple numpy
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple matplotlib
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple joblib
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple pyspark==3.1.1
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple sklearn
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple scikit-learn
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple lightgbm
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple PyArrow
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple scikit-plot
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple cython
!/emr/notebook-env/bin/pip3 install -i https://mirrors.ustc.edu.cn/pypi/web/simple pandas

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Defaul

In [2]:
import sys, random, math, os, time, glob
from datetime import datetime, date, timedelta
import pytz
from dateutil.relativedelta import *
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import logging
from collections import namedtuple, deque
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 5000)
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, StructType, StructField, IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import when, row_number, expr, concat, format_number, avg, isnan, weekofyear, rand
from pyspark.ml.feature import Bucketizer, VectorAssembler
from pyspark.ml.regression import LinearRegression as ml_LinearRegression 
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import collect_set, col, count
from pyspark.ml.fpm import FPGrowth

In [4]:
from sklearn.preprocessing import StandardScaler as sk_StandardScaler
from sklearn.decomposition import PCA as sk_pca
from sklearn.pipeline import Pipeline as sk_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import OneHotEncoder as sk_OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

from pyspark.ml import Pipeline as ml_pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler as ml_StandardScaler
from pyspark.ml.feature import PCA as ml_pca
from pyspark.ml.classification import LogisticRegression as ml_LogisticRegression

In [5]:
from lightgbm import LGBMClassifier
from sklearn.inspection import permutation_importance

In [6]:
from pyspark.sql import SparkSession
import os
os.environ["SPARK_CONF_DIR"] = "/usr/lib/spark/conf"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["SPARK_HOME"] = "/usr/lib/spark"

#spark = SparkSession.builder.appName("test3").getOrCreate()\
#spark = SparkSession.builder.appName("local[*]").master("local[*]")\
spark = SparkSession.builder.master("yarn").appName("test")\
    .config("spark.executor.instances", "12")\
    .config("spark.executor.cores", "2")\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "100g")\
    .enableHiveSupport().getOrCreate()
#     .config("spark.jars.packages", "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc1")\
#     .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")\
#     # .config('spark.local.dir', 'SOME/DIR/WHERE/YOU/HAVE/SPACE')
    # .config("spark.sql.parquet.writeLegacyFormat", true)\
    #spark = SparkSession.builder.appName("test1")\

spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNhonemptyLocation", "true")
spark.conf.set("hive.exec.dynamic.partition", "true");
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");

In [7]:
sc=spark.conf
spark
spark.sparkContext._conf.get('spark.driver.memory')
spark.sparkContext._conf.get('spark.executor.memory')
spark.sparkContext._conf.get('spark.num.executors')
spark.sparkContext._conf.get('spark.executor.cores')

'2'

In [8]:
# spark.sql('drop database IF EXISTS dwd')
# spark.sql('drop database IF EXISTS dim')
spark.sql('drop table IF EXISTS dwd.dwd_sales_order_main_flow')
spark.sql('drop table IF EXISTS dwd.dwd_sales_order_detail_flow')
spark.sql('drop table IF EXISTS dwd.dwd_sales_order_return_main')
spark.sql('drop table IF EXISTS dwd.dwd_sales_order_return_detail')
spark.sql('drop table IF EXISTS dwd.dwd_event')
spark.sql('drop table IF EXISTS dim.dim_agreement_coupon_info')
spark.sql('drop table IF EXISTS dwd.dwd_agreement_coupon_ver')
spark.sql('drop table IF EXISTS dim.dim_party_user_all')
spark.sql('drop table IF EXISTS dim.dim_party_shop')
spark.sql('drop table IF EXISTS dim.dim_item_sku')
spark.sql('drop table IF EXISTS dim.dim_party_number_kid')
spark.sql('drop table IF EXISTS dim.dim_public_date_calendar')
spark.sql('drop table IF EXISTS dim.dim_public_location_area')

DataFrame[]

In [9]:
spark.sql('create database IF NOT EXISTS dwd')
spark.sql('create database IF NOT EXISTS dim')

DataFrame[]

In [10]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_sales_order_main_flow`(`type_name` STRING,`type_value` STRING,`main_order_code` STRING, `source_channel` STRING, \
`refining_channel` STRING, `source_type` STRING, `pay_time` TIMESTAMP, `order_state` STRING, `order_state_note` STRING, \
`openid` STRING, `shopperid` STRING, `member_code` STRING, `user_phone_no` STRING, `user_phone_take_no` STRING,\
`consignee_info_code` STRING, `lego_shop_code` STRING, `dealers_shop_code` STRING, `order_order_state` STRING, \
`order_province` STRING, `order_city` STRING, `order_county` STRING, `order_address` STRING, `pay_polytype` STRING,\
`is_special_sales` STRING, `special_sales_type` STRING, `is_gwp` STRING, `receivable_amount` DOUBLE, `receipts_amount` DOUBLE, \
`sales_num` DOUBLE, `discount_amount` DOUBLE, `freight_amount` DOUBLE, `operator_code` STRING, `note` STRING, `create_time` TIMESTAMP, \
`update_time` TIMESTAMP, `achieve_time` TIMESTAMP, `delivery_time` TIMESTAMP, `scheduling_delivery_time` TIMESTAMP, \
`cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627545806',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_sales_order_main_flow'")

DataFrame[]

In [11]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_sales_order_detail_flow`(`type_name` STRING,`type_value` STRING,`source_channel` STRING, `refining_channel` STRING, \
`main_order_code` STRING, `detail_order_code` STRING, `source_type` STRING, `pay_time` TIMESTAMP, `order_state` STRING, \
`order_state_note` STRING, `openid` STRING, `opperid` STRING, `member_code` STRING, `mobile` STRING, `lego_shop_code` STRING,\
`dealers_shop_code` STRING, `item_five_lego_code` STRING, `sku_name` STRING, `is_gwp` STRING, `gwp_type` STRING, \
`item_unit_actual_amount` DOUBLE, `item_unit_standard_amount` DOUBLE, `receivable_amount` DOUBLE, `receipts_amount` DOUBLE, \
`sales_num` DOUBLE, `discount_amount` DOUBLE, `note` STRING, `create_time` TIMESTAMP, `update_time` TIMESTAMP,\
`completion_ime` TIMESTAMP, `delivery_time` TIMESTAMP, `advance_delivery_time` TIMESTAMP, `order_rrp_amount` DOUBLE, `member_detail_id` STRING, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627547702',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_sales_order_detail_flow'")

DataFrame[]

In [12]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_sales_order_return_main`(`type_name` STRING,`type_value` STRING,`return_order_code` STRING, `main_order_code` STRING, \
`source_channel` STRING, `refining_channel` STRING, `source_type` STRING, `pay_time` TIMESTAMP, `return_order_state` STRING,\
`return_order_type` STRING, `openid` STRING, `shopperid` STRING, `member_code` STRING, `mobile` STRING, `lego_shop_code` STRING,\
`lego_shop_name` STRING, `dealers_shop_code` STRING, `order_state` STRING, `order_province` STRING, `order_city` STRING,\
`order_county` STRING, `order_address` STRING, `receivable_amount` DOUBLE, `paid_in_amount` DOUBLE, `return_amount` DOUBLE,\
`return_num` BIGINT, `create_time` TIMESTAMP, `return_time` TIMESTAMP, `update_time` TIMESTAMP, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627547764',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_sales_order_return_main'")

DataFrame[]

In [13]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_sales_order_return_detail`(`type_name` STRING,`type_value` STRING,`return_ordera_detail_code` STRING, `detail_order_code` STRING, \
`main_order_code` STRING, `return_order_code` STRING, `source_channel` STRING, `refining_channel` STRING, `source_type` STRING, \
`pay_time` TIMESTAMP, `return_order_state` STRING, `return_order_type` STRING, `openid` STRING, `shopperid` STRING, `member_code` STRING, \
`mobile` STRING, `lego_shop_code` STRING, `lego_shop_name` STRING, `dealers_shop_code` STRING, `item_five_lego_code` STRING,\
`lego_sku_name` STRING, `is_gwp` STRING, `gwp_type` STRING, `receivable_amount` DOUBLE, `paid_in_amount` DOUBLE, `return_amount` DOUBLE,\
`return_num` BIGINT, `note` STRING, `return_time` TIMESTAMP, `create_time` TIMESTAMP, `update_time` TIMESTAMP, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627548594',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_sales_order_return_detail'")

DataFrame[]

In [14]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_agreement_coupon_info`(`coupon_code` STRING, `coupon_name` STRING, `coupon_type` STRING,\
`coupon_type_name` STRING, `mutex_other` STRING, `mutex_self` STRING, `coupon_priority` STRING, `reduce_condition_type` STRING,\
`reduce_condition_value` STRING, `reduce_action_type` STRING, `reduce_value` STRING, `max_reduce_value` STRING, `valid_start_type` STRING,\
`valid_delay_days` TIMESTAMP, `valid_start_time` TIMESTAMP, `valid_end_time` TIMESTAMP, `valid_time_length` DOUBLE, `valid_time_unit` STRING,\
`coupon_definition_status` STRING, `total_limit` DOUBLE, `sent_count` DOUBLE, `collected_count` DOUBLE, `redeemed_count` DOUBLE, \
`provide_type` STRING, `trigger_event` STRING, `max_collect_by_member` DOUBLE, `facevalue_per_unit` DOUBLE, `fairvalue_per_unit` DOUBLE, \
`cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627556713',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_agreement_coupon_info'")

DataFrame[]

In [15]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_agreement_coupon_ver`(`type_name` STRING,`type_value` STRING,`user_phone_no` STRING, `coupon_code` STRING, `crm_member_id` STRING,\
`crm_coupon_benefit_type` STRING, `crm_coupon_order_type` STRING, `benefit_id` STRING, `verification_id` STRING, `coupon_type` STRING, \
`order_shop` STRING, `coupon_scope` STRING, `coupon_scope_type` STRING, `channe_source` STRING, `channe_source_name` STRING,\
`source_campaign_code` STRING, `source_campaign_name` STRING, `order_time` TIMESTAMP, `order_num` BIGINT, `benefit_account_type_code` STRING,\
`benefit_account_type_name` STRING, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627557103',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_agreement_coupon_ver'")

DataFrame[]

In [16]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_party_shop`(`lego_shop_code` STRING, `lego_shop_name` STRING, `channel_source` STRING,\
`channel` STRING, `customer_code` STRING, `distributor_name`STRING, `lego_store_code` STRING, `dealer_name` STRING, \
`state_name` STRING, `region` STRING, `province_cn` STRING, `city_cn` STRING, `shop_address` STRING, `is_close` STRING,\
`open_date` DATE, `shop_area` STRING, `contact` STRING, `contact_phone_no` STRING, `area` DOUBLE, `zip_code` STRING,\
`first_sales_time` TIMESTAMP, `close_time` TIMESTAMP, `jurisdiction` STRING, `note` STRING, `up_to_amount` DOUBLE,\
`order_freight_amount_limit` DOUBLE, `partner_name` STRING, `sold_to` STRING, `ship_to` STRING, `trad_id` STRING, \
`review_channel_name` STRING, `sub_distributor_name` STRING, `sub_distributor_lvl1_name` STRING, `sub_distributor_lvl2_name` STRING, \
`sub_distributor_lvl3_name` STRING, `distributor_group` STRING, `distributor_group_name` STRING, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627551496',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_party_shop'")

DataFrame[]

In [17]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_item_sku`(`year_version` STRING, `lego_sku_id` STRING, `material_id` STRING, \
`sku_name` STRING, `sku_price_code` STRING, `rsp` DOUBLE, `new_sku_start_time` TIMESTAMP, `new_sku_end_time` TIMESTAMP,\
`sku_version_id` STRING, `version_num` STRING, `sku_specifications_id` STRING, `sku_specifications_name` STRING, \
`product_reveal_date` TIMESTAMP, `product_status` STRING, `sku_name_en` STRING, `sku_name_cn` STRING, `global_launch_date` TIMESTAMP, \
`global_delete_date` TIMESTAMP, `is_bu_cn_launch` STRING, `bu_cn_launch_date` TIMESTAMP, `bu_cn_delete_date` TIMESTAMP, \
`cn_lcs_launch_date` TIMESTAMP, `cn_ldc_launch_date` TIMESTAMP, `cn_tm_launch_date` TIMESTAMP, `cn_jd_b2b_launch_date` TIMESTAMP,\
`cn_jd_pop_launch_date` TIMESTAMP, `age_mark` STRING, `case_pack` STRING, `sku_unit` STRING, `lbs_size` DOUBLE, `sku_weight` DOUBLE, \
`sku_length` DOUBLE, `sku_width` DOUBLE, `sku_height` DOUBLE, `member_price` DOUBLE, `cost_price` DOUBLE, `stock_price` DOUBLE, \
`pricing_manner` STRING, `topic` STRING, `topic_name` STRING, `topic_group` STRING, `top_theme` STRING, `top_theme_name` STRING,\
`core_line` STRING, `is_bu_d2c_launch` STRING, `bu_d2c_launch_date` TIMESTAMP, `bu_d2c_delete_date` TIMESTAMP,\
`exclusive_products` STRING, `super_segment` STRING, `item_cn_line` STRING, `product_class` STRING, `is_batteries_included` STRING,\
`piece_count` DOUBLE, `product_ean_code` STRING, `carton_ean_code` STRING, `new_List_price_pricelist_ab_china` STRING,\
`new_ldc_extended_line_list_price` STRING, `material_group_name` STRING, `material_group` STRING, `product_sales_category` STRING,\
`sub_catergory` STRING, `fr_theme` STRING, `prod_category` STRING, `cn_tm_product_substatus` STRING, `cn_jd_b2b_product_substatus` STRING,\
`cn_jd_pop_product_substatus` STRING, `cn_jd_substatus` STRING, `in_and_out` STRING, `product_bu` STRING, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627556215',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_item_sku'")

DataFrame[]

In [18]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_party_number_kid`(`user_phone_no` STRING, `crm_member_id` STRING, `kid_id` STRING,\
`kid_name` STRING, `kid_idno` STRING, `kid_birthday` DATE, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627553796',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_party_number_kid'")

DataFrame[]

In [19]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_public_date_calendar`(`calendar_id` STRING, `date_name` STRING, `lunar_date_name` STRING,\
`year` STRING, `month` STRING, `year_month` STRING, `quarter` STRING, `quarter_name` STRING, `month_fist_calendar_id` STRING, \
`month_fist_day` STRING, `month_last_calendar_id` STRING, `month_last_day` STRING, `days_in_month` BIGINT, `weeks` BIGINT, \
`last_week_days` STRING, `last_year_days` STRING, `week_first_days` STRING, `week_last_days` STRING, `weeks_num` BIGINT,\
`weeks_name` STRING, `year_day_num` BIGINT, `year_day_weeks` BIGINT, `holiday_flag` STRING, `half_year` STRING, `if_month_lastday` STRING, \
`calendar_holiday_name` STRING, `lunar_holiday_name` STRING, `constellation` STRING, `chinese_zodiac` STRING, `lunar_year` STRING, \
`lunar_month` STRING, `lunar_month_run` STRING, `lunar_day` STRING, `solar_term_name` STRING, `lego_day` BIGINT, `lego_week` BIGINT, \
`lego_month` BIGINT, `lego_year` BIGINT, `cdp_modify_time` TIMESTAMP, `pt` STRING)\
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627553826',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_public_date_calendar'")

DataFrame[]

In [20]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_public_location_area`(`area_code` STRING, `province` STRING, `city` STRING, `telecom_operator` STRING) \
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627557721',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_public_location_area'")

DataFrame[]

In [21]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dim`.`dim_party_user_all`(`register_phone_no` STRING,\
`is_member` STRING, `is_shopper` STRING, `is_consumber` STRING, `member_card` STRING, `off_member_id` STRING,\
`crm_member_id` STRING, `wechat_app_openid` STRING, `tmall_member_id` STRING, `jd_member_id` STRING,\
`jd_shopperid` STRING, `tmall_shopperid` STRING, `wechat_unionid` STRING, `wechat_app_nickname` STRING,\
`tmall_nickname` STRING, `jd_nickname` STRING, `wechat_app_user_name` STRING, `tmall_user_name` STRING,\
`jd_user_name` STRING, `off_user_name` STRING, `gender` STRING, `birthday` DATE, `age_cohorts` STRING,\
`native_place` STRING, `family_income` BIGINT, `age` BIGINT, `state` STRING, `province` STRING, \
`city` STRING, `city_grade` STRING, `county` STRING, `street` STRING, `delivery_address` STRING,\
`marital_status` STRING, `chinese_zodiac` STRING, `constellation` STRING, `is_kid` STRING,\
`kid_num` BIGINT, `kid_gender_1` STRING, `kid_birthday_1` DATE, `kid_age_1` BIGINT, \
`kid_age_cohorts_1` BIGINT, `kid_age_learn_phase_1` BIGINT, `kid_gender_2` STRING, `kid_birthday_2` DATE, \
`kid_age_2` BIGINT, `kid_age_cohorts_2` BIGINT, `kid_age_learn_phase_2` BIGINT, `kid_gender_3` STRING, \
`kid_birthday_3` DATE, `kid_age_3` BIGINT, `kid_age_cohorts_3` BIGINT, `kid_age_learn_phase_3` BIGINT, \
`is_consent_letter` STRING, `is_touch_crm` STRING, `first_buy_date` DATE, `all_channel_first_day_num_today` BIGINT,\
`all_channel_first_day_num_crm` BIGINT, `last_buy_date` DATE, `all_channel_last_day_num_pos` BIGINT, \
`all_channel_his_buy_avg_day_num` BIGINT, `crm_usable_coupon` BIGINT, `coupon_preference_verification_m` BIGINT,\
`his_add_convert_point_preference_m` BIGINT, `about_to_past_due_point_num_Xm` BIGINT, \
`wechat_app_member_grade` STRING,`pos_member_grade` STRING, `jd_member_grade` STRING, \
`tmall_member_grade` STRING, `is_wechat_app_member` STRING, `wechat_app_member_register_date` DATE,\
`wechat_app_member_register_age_month` BIGINT, `is_tmall_member` STRING, `tmall_member_register_date` DATE,\
`tmall_member_register_age_month` BIGINT, `tmall_buy_first_time` DATE, `tmall_buy_last_time` DATE,\
`tmall_buy_last_register_day_num` BIGINT, `tmall_buy_first_register_day_num` BIGINT, \
`tmall_his_buy_avg_day_num` BIGINT, `tmall_buy_first_day_num_today` BIGINT, `tmall_buy_last_day_num_today` BIGINT,\
`is_jd_member` STRING, `jd_member_register_date` DATE, `jd_member_register_age_month` BIGINT, \
`jd_buy_first_time` DATE, `jd_buy_last_time` DATE, `jd_buy_last_register_day_num` BIGINT, \
`jd_buy_first_register_day_num` BIGINT, `jd_his_buy_avg_day_num` BIGINT, `jd_buy_first_day_num_today` BIGINT,\
`jd_buy_last_day_num_today` BIGINT, `is_crm_member` STRING, `crm_member_starts` STRING, \
`crm_member_register_date` DATE, `crm_member_register_age_month` BIGINT, `is_inside_staff` STRING, \
`is_blacklist` STRING, `is_dealer_member` STRING, `pos_memeber_register_date` DATE,\
`pos_member_register_age_month` BIGINT, `pos_buy_first_time` DATE, `pos_buy_last_time` DATE, \
`pos_buy_last_register_day_num` BIGINT, `pos_buy_first_register_day_num` BIGINT, \
`pos_his_buy_avg_day_num` BIGINT, `pos_buy_first_day_num_today` BIGINT, `pos_buy_last_day_num_today` BIGINT,\
`crm_identity_type` STRING, `crm_identity_code` STRING, `crm_usable_point` BIGINT,\
`user_email` STRING, `zip_code` STRING, `educational_status` STRING, `user_job` STRING, \
`monthly_pay` BIGINT, `hobbies_interests` STRING, `member_grade_code` STRING, `member_grade_name` STRING, \
`ly_associated_rate` STRING, `tm_associated_rate` STRING, `jd_associated_rate` STRING, \
`his_past_due_all_point` BIGINT, `all_buy_packages_num` BIGINT, `all_buy_order_num` BIGINT, \
`all_buy_associated_rate` STRING, `is_weekend_user` STRING, `is_festival_user` STRING,\
`is_cny_festival_user` STRING, `is_kid_festival_user` STRING, `is_holiday_festival_user` STRING, \
`is_christmas_festival_user` STRING, `is_festival_user_618` STRING, `is_festival_user_1111` STRING,\
`is_festival_user_1212` STRING, `is_super_brand_day` STRING,\
`crm_member_channel` STRING ,\
`is_dealer_member_s` STRING ,\
`dealer_member_source` STRING,\
`crm_thirdparty_channel` STRING ,\
`transaction_six_months` STRING,\
`tmall_fst_hyt_purchase` STRING ,\
`tmall_fst_binding_time` STRING ,\
`tmall_buy_purchase_cnt` BIGINT,\
`lewin_member_degree` STRING,\
`channel_name` STRING,\
`org_name` STRING,\
`member_channel_name` STRING,\
`member_org_name` STRING,`tmall_ouid` STRING,\
`cdp_modify_time` TIMESTAMP, `pt` STRING) \
PARTITIONED BY (`dt` STRING) \
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde' \
WITH SERDEPROPERTIES ( \
  'serialization.format' = '1' \
) \
STORED AS \
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat' \
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat' \
TBLPROPERTIES ( \
  'transient_lastDdlTime' = '1627550341', \
  'orc.compress' = 'NONE' \
) \
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dim.db/dim_party_user_all'")

DataFrame[]

In [22]:
spark.sql("CREATE external TABLE IF NOT EXISTS `dwd`.`dwd_event`(`eventdate` STRING, `user` STRING, `event_key` STRING, `freq` STRING, `pt` STRING) \
PARTITIONED BY (`dt` STRING)\
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.orc.OrcSerde'\
WITH SERDEPROPERTIES (\
  'serialization.format' = '1'\
)\
STORED AS\
  INPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcInputFormat'\
  OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.orc.OrcOutputFormat'\
TBLPROPERTIES (\
  'transient_lastDdlTime' = '1627557721',\
  'orc.compress' = 'NONE'\
)\
location 'hdfs://172.20.37.218:8020/user/hive/warehouse/dwd.db/dwd_event'")

DataFrame[]

In [23]:
spark.sql('msck repair table dwd.dwd_sales_order_main_flow')
spark.sql('msck repair table dwd.dwd_sales_order_detail_flow')
spark.sql('msck repair table dwd.dwd_sales_order_return_main')
spark.sql('msck repair table dwd.dwd_sales_order_return_detail')
spark.sql('msck repair table dwd.dwd_event')
spark.sql('msck repair table dim.dim_agreement_coupon_info')
spark.sql('msck repair table dwd.dwd_agreement_coupon_ver')
spark.sql('msck repair table dim.dim_party_user_all')
spark.sql('msck repair table dim.dim_party_shop')
spark.sql('msck repair table dim.dim_item_sku')
spark.sql('msck repair table dim.dim_party_number_kid')
spark.sql('msck repair table dim.dim_public_date_calendar')
spark.sql('msck repair table dim.dim_public_location_area')

DataFrame[]

In [24]:
spark.sql('show tables in dwd').show()
spark.sql('show tables in dim').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|     dwd|dwd_agreement_cou...|      false|
|     dwd|           dwd_event|      false|
|     dwd|dwd_sales_order_d...|      false|
|     dwd|dwd_sales_order_m...|      false|
|     dwd|dwd_sales_order_r...|      false|
|     dwd|dwd_sales_order_r...|      false|
+--------+--------------------+-----------+

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|     dim|dim_agreement_cou...|      false|
|     dim|        dim_item_sku|      false|
|     dim|dim_party_number_kid|      false|
|     dim|      dim_party_shop|      false|
|     dim|  dim_party_user_all|      false|
|     dim|dim_public_date_c...|      false|
|     dim|dim_public_locati...|      false|
+--------+--------------------+-----------+



In [25]:
spark.stop()